<a href="https://colab.research.google.com/github/T-Sunm/Learn-Data-Visualization-Kaggle-/blob/main/Exercise_Bar_Charts_and_Heatmaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F116573%2F3551030%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240822%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240822T155952Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D70eb6977638822c250984444519e8b64dff2b05e3af083f305b29390ca65492a38452e4c311fc80c79f5be211dca79d93fb19e623ea09327b8e11198a495a1ce981c2a8b7ba702df196578e3add849f817c99e90682149ed1f292d08d07e648fefbff47b9bbd98a48f78d50f28131ffc24bdb7c3ff4af3abbdf0068d2500d736e96e02b4491b0b90909d101e16123695bc950bc7d98614f19955b6cb764f7b6863c9c9333bfbccd8f1bf982577303fadd729347e6e3c1358254095d11062656dca370030587c5704921ec452826571ae8e8f7f493e488b2b00e51cc2f93b65789c93ede202109da7bf8e13c89004dade69c1b509af10badbc4529e575058d2b6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Data Visualization](https://www.kaggle.com/learn/data-visualization) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/bar-charts-and-heatmaps).**

---


In this exercise, you will use your new knowledge to propose a solution to a real-world scenario.  To succeed, you will need to import data into Python, answer questions using the data, and generate **bar charts** and **heatmaps** to understand patterns in the data.

## Scenario

You've recently decided to create your very own video game!  As an avid reader of [IGN Game Reviews](https://www.ign.com/reviews/games), you hear about all of the most recent game releases, along with the ranking they've received from experts, ranging from 0 (_Disaster_) to 10 (_Masterpiece_).

![ex2_ign](https://storage.googleapis.com/kaggle-media/learn/images/Oh06Fu1.png)

You're interested in using [IGN reviews](https://www.ign.com/reviews/games) to guide the design of your upcoming game.  Thankfully, someone has summarized the rankings in a really useful CSV file that you can use to guide your analysis.

## Setup

Run the next cell to import and configure the Python libraries that you need to complete the exercise.

In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

The questions below will give you feedback on your work. Run the following cell to set up our feedback system.

In [ ]:
# Set up code checking
import os
if not os.path.exists("../input/ign_scores.csv"):
    os.symlink("../input/data-for-datavis/ign_scores.csv", "../input/ign_scores.csv")
from learntools.core import binder
binder.bind(globals())
from learntools.data_viz_to_coder.ex3 import *
print("Setup Complete")

## Step 1: Load the data

Read the IGN data file into `ign_data`.  Use the `"Platform"` column to label the rows.

In [ ]:
# Path of the file to read
ign_filepath = "../input/ign_scores.csv"

# Fill in the line below to read the file into a variable ign_data
ign_data = pd.read_csv(ign_filepath, index_col = "Platform")

# Run the line below with no changes to check that you've loaded the data correctly
step_1.check()

In [ ]:
# Lines below will give you a hint or solution code
#step_1.hint()
#step_1.solution()

## Step 2: Review the data

Use a Python command to print the entire dataset.

In [ ]:
# Print the data
ign_data

The dataset that you've just printed shows the average score, by platform and genre.  Use the data to answer the questions below.

In [ ]:
# Fill in the line below: What is the highest average score received by PC games,
# for any genre?
high_score = ign_data.loc["PC"].max()

# Fill in the line below: On the Playstation Vita platform, which genre has the
# lowest average score? Please provide the name of the column, and put your answer
# in single quotes (e.g., 'Action', 'Adventure', 'Fighting', etc.)
worst_genre_index = ign_data.loc["PlayStation Vita"].argmin()
worst_genre = ign_data.columns[worst_genre_index]

# Check your answers
step_2.check()

In [ ]:
# Lines below will give you a hint or solution code
#step_2.hint()
#step_2.solution()

## Step 3: Which platform is best?

Since you can remember, your favorite video game has been [**Mario Kart Wii**](https://www.ign.com/games/mario-kart-wii), a racing game released for the Wii platform in 2008.  And, IGN agrees with you that it is a great game -- their rating for this game is a whopping 8.9!  Inspired by the success of this game, you're considering creating your very own racing game for the Wii platform.

#### Part A

Create a bar chart that shows the average score for **racing** games, for each platform.  Your chart should have one bar for each platform.

In [ ]:
racing_plat = ign_data['Racing']
print(racing_plat)

In [ ]:
# Bar chart showing average score for racing games by platform
racing_plat = ign_data['Racing']
plt.figure(figsize=(10, 6))
sns.barplot(x=racing_plat, y=ign_data.index)
plt.title('Average score for racing games by platform')
plt.xlabel('Rating')
plt.ylabel('Platforms')
# Check your answer
step_3.a.check()

In [ ]:
# Lines below will give you a hint or solution code
#step_3.a.hint()
# step_3.a.solution_plot()

#### Part B

Based on the bar chart, do you expect a racing game for the **Wii** platform to receive a high rating?  If not, what gaming platform seems to be the best alternative?

In [ ]:
#step_3.b.hint()

In [ ]:
# Check your answer (Run this code cell to receive credit!)
# step_3.b.solution()

## Step 4: All possible combinations!

Eventually, you decide against creating a racing game for Wii, but you're still committed to creating your own video game!  Since your gaming interests are pretty broad (_... you generally love most video games_), you decide to use the IGN data to inform your new choice of genre and platform.

#### Part A

Use the data to create a heatmap of average score by genre and platform.  

In [ ]:
plt.figure(figsize=(10,9))
# Heatmap showing average game score by platform and genre
sns.heatmap(data=ign_data,annot=True)
plt.title('average game score by platform and genre')
plt.xlabel('Genre')
# Check your answer
step_4.a.check()

In [ ]:
# Lines below will give you a hint or solution code
#step_4.a.hint()
step_4.a.solution_plot()

#### Part B

Which combination of genre and platform receives the highest average ratings?  Which combination receives the lowest average rankings?

In [ ]:
#step_4.b.hint()

In [ ]:
# Check your answer (Run this code cell to receive credit!)
step_4.b.solution()

# Keep going

Move on to learn all about **[scatter plots](https://www.kaggle.com/alexisbcook/scatter-plots)**!

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/data-visualization/discussion) to chat with other learners.*